# Test building load profile scenario reduction

In [ ]:
import os
os.chdir('..') # move to the root directory (from dev)

In [ ]:
import numpy as np
import pandas as pd
import itertools
from scenarioReducer import Fast_forward
from utils import scale_profile
from prob_models import shape_prior_model, level_prior_model

In [ ]:
# set up params
years = list(range(2012, 2018))
ids = [0, 4, 8, 19, 25, 40, 58, 102, 104, 118]

data_dir = os.path.join('data','processed')
building_file_pattern = 'ly_{id}-{year}.csv'

n_buildings = 8

In [ ]:
# sample vectors of building-years (scenarios)
np.random.seed(0)
n_samples = 1000
scenarios = shape_prior_model(n_buildings, n_samples, ids, years)

In [ ]:
print(scenarios)

In [ ]:
# load building-year load profiles once to reduce I/O time
load_profiles = {
    f'{building_id}-{year}': pd.read_csv(
        os.path.join(data_dir, building_file_pattern.format(id=building_id, year=year)),
        usecols=['Equipment Electric Power [kWh]']
        )['Equipment Electric Power [kWh]'].to_numpy()\
            for building_id, year in itertools.product(ids, years)
}

In [ ]:
def get_scenario_stats(building_scenario_vector, load_profiles_dict):
    """Compute mean, standard deviation, and peak of aggregate load for a given scenario,
    i.e. set of building-year profiles (profiles for each building in scenario)."""

    load_profiles = []

    for building_tuple in building_scenario_vector:
        if len(building_tuple) == 2:
            building_id,year = building_tuple
        if len(building_tuple) == 4:
            building_id,year = building_tuple[:2]
            mean,peak = building_tuple[2:]

        load_profile = load_profiles_dict[f'{int(building_id)}-{int(year)}']

        if len(building_tuple) == 4:
            load_profile = scale_profile(load_profile, mean, peak)

        load_profiles.append(load_profile)

    aggregate_load = np.sum(load_profiles, axis=0)

    return np.mean(aggregate_load), np.std(aggregate_load), np.max(aggregate_load)

In [ ]:
def rescale_array(a, invert=False, bounds=None):
    assert not (invert and bounds is None), 'bounds must be provided when inverting scaling.'
    old_bounds = (a.min(), a.max()) if not invert else (0, 1)
    new_bounds = (0, 1) if not invert else bounds
    return np.interp(a, old_bounds, new_bounds)

In [ ]:
scenario_stats = np.array([get_scenario_stats(scenario, load_profiles) for scenario in scenarios])

In [ ]:
print(scenario_stats)

In [ ]:
bounds = [(np.min(scenario_stats[:,0]), np.max(scenario_stats[:,0])),(np.min(scenario_stats[:,1]), np.max(scenario_stats[:,1])),(np.min(scenario_stats[:,2]), np.max(scenario_stats[:,2]))]
print(bounds)

In [ ]:
# rescale scenario stats to range [-1,1] for each exis
scaled_scenario_stats = np.array([rescale_array(scenario_stats[:,0]), rescale_array(scenario_stats[:,1]), rescale_array(scenario_stats[:,2])]).T

## Perform scenario reduction

In [ ]:
probs = np.ones(shape=n_samples)/n_samples # uniform probability of scenarios
num_reduced_scenarios = 10

FFreducer = Fast_forward(scaled_scenario_stats.T, probs)

In [ ]:
reduced_scenario_stats, reduced_probs, reduced_indices = FFreducer.reduce(distance=1,n_scenarios=num_reduced_scenarios)
# use 1-norm distance metric for reduction
reduced_scenario_stats = reduced_scenario_stats.T
reduced_scenarios = scenarios[reduced_indices]

In [ ]:
rescaled_reduced_scenario_stats = np.array([rescale_array(reduced_scenario_stats[:,0], invert=True, bounds=bounds[0]), rescale_array(reduced_scenario_stats[:,1], invert=True, bounds=bounds[1]), rescale_array(reduced_scenario_stats[:,2], invert=True, bounds=bounds[2])]).T

In [ ]:
# check reduced scenarios have been accessed correctly
assert np.isclose(rescaled_reduced_scenario_stats, scenario_stats[reduced_indices]).all()

In [ ]:
print(rescaled_reduced_scenario_stats)

In [ ]:
print(scenarios[reduced_indices])

## Test modulised version

In [ ]:
from load_scenario_reduction import reduce_load_scenarios

In [ ]:
module_reduced_scenarios, reduced_probs = reduce_load_scenarios(scenarios, load_profiles, num_reduced_scenarios)

In [ ]:
print(module_reduced_scenarios)
print(reduced_probs)

In [ ]:
assert np.isclose(module_reduced_scenarios, reduced_scenarios).all()